## LLM chat completions

In [1]:
from openai import OpenAI
import json
from rich import print

import dotenv
dotenv.load_dotenv()

False

In [2]:
client = OpenAI()

In [3]:
messages = [{"role": "user", "content": "What's the weather like in London?"}]
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

In [4]:
print(response)

ChatCompletion(
    id='chatcmpl-9Vyfey5pi1USHJuOUVgZ1VPjBCq6a',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content="I'm unable to provide real-time updates as my current capabilities don't include accessing
live data. For the latest weather information in London, I suggest checking a reliable weather website or app such 
as the BBC Weather website, the Met Office website, or using a weather app on your smartphone.",
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717408414,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=55, prompt_tokens=14, total_tokens=69)
)

In [6]:
def get_current_weather(location):
    """Get the current weather in a given city"""
    if "london" in location.lower():
        return json.dumps({"temperature": "20 C"})
    elif "san francisco" in location.lower():
        return json.dumps({"temperature": "15 C"})
    elif "paris" in location.lower():
        return json.dumps({"temperature": "22 C"})
    else:
        return json.dumps({"temperature": "unknown"})



messages = [{"role": "user", "content": "What's the weather like in London?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [7]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response)

ChatCompletion(
    id='chatcmpl-9VyY9IkbsXAFsuot3FpOHpYtPZmak',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_ooS31KvjUQzTNJbabNbyGCwg',
                        function=Function(arguments='{"location":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717407949,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=66, total_tokens=81)
)

## Agents & Tool use

In [5]:
from agents.llms import OpenAIChatCompletion

from agents.tools import get_current_weather
from agents.tool_executor import need_tool_use

In [10]:
llm = OpenAIChatCompletion()
llm.bind_tools([get_current_weather])

In [19]:
messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)
print(output)

ChatCompletion(
    id='chatcmpl-9Vya74OCplg2FUgLc2X2N3vbmU2IT',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_Yg8j0dnFo1KS6rVHPatnIHjt',
                        function=Function(arguments='{"city":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717408071,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=83, total_tokens=98)
)

In [21]:
if need_tool_use(output):
    print("Using weather tool")
    tool_results = llm.run_tools(output)
    print(tool_results)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [
        {"role": "user", "content": "Think step by step and answer my question based on the above context."}
    ]
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

Using weather tool

[
    {
        'tool_call_id': 'call_Yg8j0dnFo1KS6rVHPatnIHjt',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"FeelsLikeC": "16", "FeelsLikeF": "61", "cloudcover": "100", "humidity": "77", 
"localObsDateTime": "2024-06-03 09:56 AM", "observation_time": "08:56 AM", "precipInches": "0.0", "precipMM": 
"0.0", "pressure": "1022", "pressureInches": "30", "temp_C": "16", "temp_F": "61", "uvIndex": "4", "visibility": 
"10", "visibilityMiles": "6", "weatherCode": "122", "weatherDesc": [{"value": "Overcast"}], "weatherIconUrl": 
[{"value": ""}], "winddir16Point": "NW", "winddirDegree": "310", "windspeedKmph": "7", "windspeedMiles": "4"}'
    }
]

Here's a step-by-step breakdown of the given weather information for London:

1. **Temperature**:
   - The temperature is 16°C (61°F).

2. **Feels Like**:
   - It feels like 16°C (61°F), which means the actual temperature and the perceived temperature are the same.

3. **Cloud Coverage**:
   - The cloud cover is 100%, indicating that the sky is fully overcast.

4. **Humidity**:
   - The humidity level is 77%, which is relatively high.

5. **Precipitation**:
   - There is no precipitation (0.0 mm or 0.0 inches).

6. **Pressure**:
   - The atmospheric pressure is 1022 hPa (30 inches).

7. **Visibility**:
   - Visibility is 10 km (6 miles).

8. **Wind**:
   - The wind is coming from the northwest (310°) at a speed of 7 km/h (4 mph).

9. **UV Index**:
   - The UV index is 4, which is considered moderate.

10. **Weather Description**:
    - The weather is described as "Overcast."

**Summary**: Today in London, it is 16°C (61°F) and overcast with 100% cloud cover. There is no precipitation, and 
the humidity level is relatively high at 77%. The wind is mild, blowing from the northwest at 7 km/h (4 mph), and 
visibility is good at 10 km (6 miles). The atmospheric pressure is quite stable at 1022 hPa. Overall, it is a cool 
and cloudy day.

## Langchain tools

In [6]:
from agents.tool_executor import ToolRegistry, need_tool_use
from agents.tools import get_current_weather, wikipedia_search, google_search, image_inspector
import json
from langchain.tools import tool

In [7]:
@tool
def get_current_weather(city: str) -> str:
    """Get the current weather for a given city.

    Args:
      city (str): The city to fetch weather for.

    Returns:
      str: current weather condition, or None if an error occurs.
    """
    try:
        data = json.dumps({"temperature": "12c"})
        return data
    except Exception as e:
        logger.exception(e)
        error_message = f"Error fetching current weather for {city}: {e}"
        return error_message

In [8]:
registry = ToolRegistry()
registry.register_tool(get_current_weather)
print(registry.openai_tools[0])

{
    'type': 'function',
    'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a given city.\n\n    Args:\n      city (str): The city to fetch
weather for.\n\n    Returns:\n      str: current weather condition, or None if an error occurs.',
        'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}, 'required': ['city']}
    }
}

In [9]:
llm = OpenAIChatCompletion()

In [11]:
messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)
print(output)

ChatCompletion(
    id='chatcmpl-9VygJHJWpvkbPvBCY7IyMwRg1b3VH',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content="I'm sorry, but I can't provide real-time updates. To find the current weather in London, 
you can check a weather website, use a weather app on your smartphone, or ask a smart home device.",
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717408455,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=41, prompt_tokens=15, total_tokens=56)
)

In [12]:
llm.bind_tools([get_current_weather])

messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)
print(output)

ChatCompletion(
    id='chatcmpl-9VygVxHfe4jaiQJktW5hrFidaEqtq',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_pW99XhwGEvf8iDNaXIJoA66A',
                        function=Function(arguments='{"city":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717408467,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=83, total_tokens=98)
)